<a href="https://colab.research.google.com/github/hussain033/ML_Projects_base/blob/main/Classify_leaf_by_name.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Leaf Classification - 1#
###Classifies Leaf and returns its name###
###By using Mobilenetv2 pretrained Model###

##Importing Libraries##

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorboard

##Loading Data##

In [ ]:
img_height = 120
img_width = 180
batch_size = 16

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  '/content/drive/MyDrive/Colab Notebooks/Plants/train3',
  labels = 'inferred',
  seed=123,
  image_size=(img_height, img_width),
  shuffle = True,
  batch_size=batch_size)

Found 4274 files belonging to 12 classes.


In [ ]:
val_data_dir = '/content/drive/MyDrive/Colab Notebooks/Plants/valid3'
val_ds = tf.keras.utils.image_dataset_from_directory(
  val_data_dir,
  labels = 'inferred',
  seed=123,
  image_size=(img_height, img_width),
  shuffle = True,
  )

Found 110 files belonging to 12 classes.


In [ ]:
test_data_dir = '/content/drive/MyDrive/Colab Notebooks/Plants/test3'
test_ds = tf.keras.utils.image_dataset_from_directory(
  test_data_dir,
  labels = 'inferred',
  seed=123,
  image_size=(img_height, img_width),
  )

Found 110 files belonging to 12 classes.


##Pre processing Layer##

In [ ]:
norm_layer = tf.keras.Sequential([
    tf.keras.layers.Resizing(224,224),
    tf.keras.layers.Rescaling(1./255)
])

##Augmentation Layer##

In [ ]:
data_aug = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.2),
])

##Building Model##

In [ ]:
Modelnetv2 = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

In [ ]:
feature_extractor_model = Modelnetv2

In [ ]:
featureExtract_Model = hub.KerasLayer(
    feature_extractor_model,
    input_shape=(224, 224, 3),
    trainable=False)

In [ ]:
pretrained_Model = tf.keras.Sequential([
    norm_layer,
    data_aug,
    featureExtract_Model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(12)
])

In [ ]:
pretrained_Model.build((batch_size,img_height,img_width,3))

In [ ]:
pretrained_Model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (16, 224, 224, 3)         0         
                                                                 
 sequential_1 (Sequential)   (16, 224, 224, 3)         0         
                                                                 
 keras_layer (KerasLayer)    (16, 1280)                2257984   
                                                                 
 flatten (Flatten)           (16, 1280)                0         
                                                                 
 dropout (Dropout)           (16, 1280)                0         
                                                                 
 dense (Dense)               (16, 12)                  15372     
                                                                 
Total params: 2,273,356
Trainable params: 15,372
Non-t

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
optimizer = 'adam'
metrics = ['accuracy']

In [ ]:
pretrained_Model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001), loss = loss_fn, metrics = metrics)

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    mode = 'auto',
    patience = 2,
    min_delta = 0,
    verbose = 0,
    restore_best_weights = True,
)

##Training Model##

In [ ]:
pretrained_Model.fit(train_ds, epochs = 15, callbacks = [early_stopping],
          validation_data = val_ds)

##Testing and Evaluating Model##

In [ ]:
test_loss, test_accuracy = pretrained_Model.evaluate(test_ds)

4/4 [==============================] - 20s 690ms/step - loss: 0.6646 - accuracy: 0.9182


In [ ]:
pretrained_Model.save('/content/drive/MyDrive/AI projects')